# Import Kaggle and Download data

# Let's get the fun started

In [1]:
# installing and upgrading the "kaggle" package
!pip install kaggle
!pip install --upgrade --force-reinstall --no-deps kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.16-py3-none-any.whl size=110685 sha256=45e0579495e45e3498abec88f9a1187dbe9fde79f22b96c1dd62393553ef5393
  Stored in directory: /root/.cache/pip/wheels/43/4b/fb/736478af5e8004810081a06259f9aa2f7c3329fc5d03c2c412
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.16
    Uninstalling kaggle-1.5.16:
      Successfully uninstalled kaggle-1.5.16


In [2]:
# Uploading the configuration file to authenticate Kaggle API access
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"solafafadlallah","key":"4d164dae88f65e5ac8385268e4e8016f"}'}

In [3]:
# configuring the Kaggle API authentication
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

rm: cannot remove '/root/.kaggle': No such file or directory


In [4]:
# downloading the data files related to the "kernel-methods-ammi-2023" competition to the current working directory
!kaggle competitions download -c kernel-methods-ammi-2023

 99% 1.00M/1.01M [00:00<00:00, 1.50MB/s]
100% 1.01M/1.01M [00:00<00:00, 1.52MB/s]


In [5]:
#extracting the files
!unzip /content/kernel-methods-ammi-2023.zip

Archive:  /content/kernel-methods-ammi-2023.zip
  inflating: Xte0.csv                
  inflating: Xte0_mat100.csv         
  inflating: Xte1.csv                
  inflating: Xte1_mat100.csv         
  inflating: Xte2.csv                
  inflating: Xte2_mat100.csv         
  inflating: Xtr0.csv                
  inflating: Xtr0_mat100.csv         
  inflating: Xtr1.csv                
  inflating: Xtr1_mat100.csv         
  inflating: Xtr2.csv                
  inflating: Xtr2_mat100.csv         
  inflating: Ytr0.csv                
  inflating: Ytr1.csv                
  inflating: Ytr2.csv                


In [7]:
# importing the neccessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import itertools
import seaborn as sns
from collections import defaultdict
import cvxopt
from itertools import product
from scipy import optimize
!pip install cvxopt
!pip install qpsolvers
from qpsolvers import solve_qp
from sklearn.base import BaseEstimator, ClassifierMixin
import timeit
from sklearn.model_selection import cross_validate
from numpy.lib.function_base import iterable
from scipy.special import expit as sigmoid

Obtaining the data

In [8]:
#1st Data Set
file_path = "/content/Xtr0.csv"
Xtr0 = np.array(pd.read_csv(file_path, sep=",", index_col=0))
file_path = "/content/Ytr0.csv"
Ytr0 = 2*(np.array(pd.read_csv(file_path, index_col=0)).reshape(Xtr0.shape[0]))-1
file_path = "/content/Xte0.csv"
Xte0 = np.array(pd.read_csv(file_path, sep=",", index_col=0))


#2nd Data Set
file_path = "/content/Xtr1.csv"
Xtr1 = np.array(pd.read_csv(file_path, sep=",", index_col=0))
file_path = "/content/Ytr1.csv"
Ytr1 = 2*(np.array(pd.read_csv(file_path, index_col=0)).reshape(Xtr1.shape[0]))-1
file_path = "/content/Xte1.csv"
Xte1 = np.array(pd.read_csv(file_path, sep=",", index_col=0))


#3rd Data Set
file_path = "/content/Xtr2.csv"
Xtr2 = np.array(pd.read_csv(file_path, sep=",", index_col=0))
file_path = "/content/Ytr2.csv"
Ytr2 = 2*(np.array(pd.read_csv(file_path, index_col=0)).reshape(Xtr2.shape[0]))-1
file_path = "/content/Xte2.csv"
Xte2 = np.array(pd.read_csv(file_path, sep=",", index_col=0))


In [9]:
def corspond(Seq, l):
  '''
  This function generates all possible sequences of length l using elements from Seq and creates
  a dictionary that maps each sequence to its corresponding index.
  '''
  correspond = {}
  cont = 0

  for i in product(Seq, repeat = l):
    j = ''
    for m in range(l):
        j = j + i[m]
    correspond[j] = cont
    cont = cont + 1
  return correspond

In [10]:
def mismatch(l, n, correspond):

  a = 4**l
  b = np.zeros((a,a))
  sequnce = list(correspond.keys())
  for i in range(a):
    sequnce1 = sequnce[i]
    for j in range(a):
      sequnce2 = sequnce[j]
      ms = 0
      cont = 0
      while (ms <= n) and (cont <= l-1):
        if sequnce1[cont] != sequnce2[cont]:
          ms = ms + 1
        cont = cont + 1
      if ms <= n:
          b[i,j] = 1
  return b

In [11]:
def Gram_mis_match(X, Xtest, corspond, l, mis_match):
  '''
  X: training dataset
  X_test: test dataset
  corspond: a dictionary that maps each sequence to its corresponding index
  mismatch : mismatch matrix
  '''
  X_new = np.concatenate((X,Xtest), axis=0)
  vect = np.zeros((X_new.shape[0],4**l))
  for s in range(X_new.shape[0]):
    for j in range((len(X_new[0][0]))-l+1):
      var = X_new[s][0][j:j+l]
      vect[s] = vect[s] + mis_match[corspond[var]]
    vect[s]/=np.sqrt(np.dot(vect[s],vect[s]))

  return np.dot(vect,vect.T)


#Kernel Normalization

In [12]:
def Kernel_Norm(K):
  K_norm = K.copy()
  for i in range(K.shape[0]):
    for j in range(K.shape[1]):
      K_norm[i,j] /= (K[i,i]*K[j,j])**(0.5)
  return K_norm

#kernel Ridge regression gives a 66.6% accurcy!

# Ridge Regression

In [13]:
class Ridge_regression():

  """
    This class implements the kernel ridge regression method for mismatch kernels.
  """

  def __init__(self, lmbd = 1.0, solver='closed_form', cores=None, k_mer = 6, mis_match_mat = None):
    """
      - lmbd: Regularization parameter
      - cores: mismatch correspondence matrix
      - k_mer: k-mer length (Length of the sequence)
      - mis_match_mat: The matrix of mismatches among all k-length sequences
    """
    self.lmbd = lmbd
    self.cores = cores
    self.k_mer = k_mer
    self.mis_match_mat = mis_match_mat

  def fit(self, Xtr, Ytr, Xte):

    """
      Xtr: Training data
      Ytr: Target values
      Xte: Test data
    """
    self.Xtr = Xtr
    self.Ytr = Ytr
    self.Xte = Xte

    N = self.Xtr.shape[0]

    Reg_mat = self.lmbd*np.identity(N) #diagonal matrix with the regularization parameter lambda

    K_mismatch = Gram_mis_match(self.Xtr, self.Xte, self.cores, self.k_mer, self.mis_match_mat)


    K_mismatch =Kernel_Norm(K_mismatch)

    split = N

    self.K_mismatch_tr = K_mismatch[:split, :split]

    self.K_mismatch_te = K_mismatch[split:, :split]

    Bita = np.linalg.inv((self.K_mismatch_tr + Reg_mat)).dot(Ytr)

    self.Bita = Bita


  def pred(self):
    Bita = self.Bita
    K_te = self.K_mismatch_te.T
    K_pred = np.dot(Bita, K_te)

    self.K_pred = K_pred

    return self.K_pred


#Training the Data:

In [14]:
print('training first data set')
sequence =['A', 'C', 'G', 'T'] #nucleotides: adenine, cytosine, guanine, and thymine
k_mer = 7
m = 1
#m is the number of allowed mismatches in the sequences. It defines the maximum number of positions at which two sequences can differ and still be considered similar

correspond = corspond(sequence, k_mer)
mismatch_mat = mismatch(k_mer, m, correspond)
lmbd = 1

RR = Ridge_regression(lmbd = 1, cores = correspond, k_mer = k_mer, mis_match_mat = mismatch_mat)

RR.fit(Xtr0, Ytr0, Xte0)

y_pred = (np.sign(RR.pred())).astype("int8")

y_pred_df0 = pd.DataFrame({"id": range(1000), "Bound": y_pred.T})



print('training second data set')



sequence = ['A', 'C', 'G', 'T'] #nucleotides: adenine, cytosine, guanine, and thymine
k_mer = 7
m = 1
RR = Ridge_regression(lmbd = 1, cores = correspond, k_mer = k_mer, mis_match_mat = mismatch_mat)

RR.fit(Xtr1, Ytr1, Xte1)

y_pred = (np.sign(RR.pred())).astype("int8")

y_pred_df1 = pd.DataFrame({"id": range(1000), "Bound": y_pred.T})



print('training third data set')



sequence = ['A', 'C', 'G', 'T'] #nucleotides: adenine, cytosine, guanine, and thymine
k_mer = 7
m = 3
RR = Ridge_regression(lmbd = 0.07, cores = correspond, k_mer = k_mer, mis_match_mat = mismatch_mat)

RR.fit(Xtr2, Ytr2, Xte2)

y_pred = (np.sign(RR.pred())).astype("int8")

y_pred_df2 = pd.DataFrame({"id": range(1000), "Bound": y_pred.T})

training first data set
training second data set
training third data set


###Submitting the results

In [ ]:
df_pred = pd.concat([y_pred_df0,y_pred_df1,y_pred_df2])
df_pred = df_pred.replace(-1,0)
df_pred.to_csv('Y_pred_RR', index = False)

#let's try to normalize the kernels

#Kernel Normalization

In [7]:
def Kernel_Norm(K):
  K_norm = K.copy()
  for i in range(K.shape[0]):
    for j in range(K.shape[1]):
      K_norm[i,j] /= (K[i,i]*K[j,j])**(0.5)
  return K_norm

#Lets do logistic regression

Before training

In [6]:
sequence =['A', 'C', 'G', 'T'] #nucleotides: adenine, cytosine, guanine, and thymine
k_mer = 7
m = 1
#m is the number of allowed mismatches in the sequences. It defines the maximum number of positions at which two sequences can differ and still be considered similar

correspond = corspond(sequence, k_mer)
mismatch_mat = mismatch(k_mer, m, correspond)

#Here we do the SVM

#Utilities

In [101]:
import cvxopt

def cvxopt_qp(P, q, G, h, A, b):
    P = .5 * (P + P.T)
    cvx_matrices = [
        cvxopt.matrix(M) if M is not None else None for M in [P, q, G, h, A, b]
    ]
    #cvxopt.solvers.options['show_progress'] = False
    solution = cvxopt.solvers.qp(*cvx_matrices, options={'show_progress': False})
    return np.array(solution['x']).flatten()

solve_qp = cvxopt_qp

In [108]:
def svm_dual_soft_to_qp_kernel(K, y, C=1):
    n = K.shape[0]
    assert (len(y) == n)

    # Dual formulation, soft margin
    P = y[None]*K*y[:,None]
    # As a regularization, we add epsilon * identity to P
    eps = 1e-12
    P += eps * np.eye(n)
    q = - np.ones(n)
    G = np.vstack([-np.eye(n), np.eye(n)])
    h = np.hstack([np.zeros(n), C * np.ones(n)])
    A = y[np.newaxis, :]
    b = np.array([0.])
    return P, q, G, h, A, b

In [109]:
class KernelSVM():
    '''
    Kernel SVM Classification

    Methods
    ----
    fit
    predict
    '''
    def __init__(self, C=0.1, cores = None, k_mer = 6 ,mis_match_mat = None, **kwargs):

        super().__init__(**kwargs)
        self.mis_match_mat = mis_match_mat
        self.cores = cores
        self.k_mer = k_mer
        self.lbda = C

    def fit_K(self, Xtr, Xte, y, tol=1e-3):
        # Solve dual problem
        self.Xtr = Xtr
        self.Xte = Xte
        #self.lbda = lbda


        N = self.Xtr.shape[0]


        K_mismatch = Gram_mis_match(self.Xtr, self.Xte, self.cores, self.k_mer, self.mis_match_mat)
        self.K_mismatch = K_mismatch

        split = N

        self.K_mismatch_tr = K_mismatch[:split, :split]

        self.K_mismatch_te = K_mismatch[split:, split:]

        self.alpha = solve_qp(*svm_dual_soft_to_qp_kernel(self.K_mismatch_tr, y, C=self.lbda))
        y = 2 * y - 1

        n = len(y)

        q = - y.astype(float)
        P = self.K_mismatch_tr
        G = np.zeros((2 * n, n))
        G[:n, :] = - np.diag(y)
        G[n:, :] = np.diag(y)
        h = np.zeros(2 * n)
        h[n:] = 1 / (2 * self.lbda * n)
        #self.alpha = qpsolvers.solve_qp(P, q, G, h, solver='cvxopt')

        # Compute support vectors and bias b
        sv = np.logical_and((self.alpha > tol), (self.lbda - self.alpha > tol))
        self.bias = y[sv] - self.K_mismatch_tr[sv].dot(self.alpha * y)
        self.bias = self.bias.mean()

        self.support_vector_indices = np.nonzero(sv)[0]
        self.fitted_ = True

        return self

    def decision_function_K(self, K_x):
        K_x = self.K_mismatch_te
        return K_x@(self.alpha[:len(self.K_mismatch_te)])

    def predict(self):
        print(self.alpha.shape, self.K_mismatch_te.shape, self.bias)
        return np.sign(self.decision_function_K(self.K_mismatch_te))

In [23]:
sequence =['A', 'C', 'G', 'T'] #nucleotides: adenine, cytosine, guanine, and thymine
k_mer = 7
m = 1
#m is the number of allowed mismatches in the sequences. It defines the maximum number of positions at which two sequences can differ and still be considered similar

correspond = corspond(sequence, k_mer)
mismatch_mat = mismatch(k_mer, m, correspond)

In [110]:
SVM = KernelSVM(C=0.1, cores = correspond, k_mer = k_mer, mis_match_mat = mismatch_mat)

In [111]:
import qpsolvers

In [112]:
lbda = 0.1

In [ ]:
SVM.fit_K(Xtr0, Xte0, Ytr0)

In [ ]:
def change_sign(x):
    return 2 * x - 1

In [ ]:
def conv_to_binary(x):
    return ((x + 1) / 2).astype(int)

In [ ]:
class Logis_Regression(BaseEstimator, ClassifierMixin):
    """
    by using BaseEstimator and ClassifierMixin, the Logis_regression class becomes a scikit-learn-compatible classifier,
    which means it can be easily integrated into scikit-learn pipelines, work well with other scikit-learn components,
    and leverage the tools and utilities provided by scikit-learn to streamline the machine learning workflow.
    """
    def __init__(self, iter_max=20, tol=1E-6, lmbd=1.0):

        self.lmbd = lmbd
        self.tol = tol
        self.iter_max = iter_max

    @property
    def PWISE(self):
        return True

    def fit(self, Ker, y):
        """
        The fit method of the class, used for training the Kernel Logistic Regression model.

        - K is the kernel matrix
        - y is the target labels.
        """
        param = np.zeros(len(Ker)) #variable as an array of zeros with the same length as the Kernel matrix
        y = change_sign(y)

        for i in range(self.iter_max):
            Prob_t = - sigmoid(- y * (Ker @ param)) #Computing the sigmoid for each entry of the vector
            Weight_t = sigmoid(Ker @ param) * sigmoid(-Ker @ param) #calculating the weight for each sample using the logistic sigmoid function.
            Inter = Ker @ param - Prob_t * y / Weight_t #An intermediate variable which is used in the optimization process.

            param_update = K_RR(Ker, Inter, Weight_t, lbda=self.lbda)

            d = np.linalg.norm(param_update - param)
            param = param_update

            if d < self.tol:
                break

        self.param_ = param
        self.fitted_ = True
        self.Ker_fit_ = Ker

        return self

    def predict(self, Ker):
        return conv_to_binary(np.sign(Ker @ self.param_))

#Implementing the TRIENODE

In [ ]:
# A Trie is a tree-like data structure used for efficient storage and retrieval of strings.
# Each node in the Trie represents a single character, and the path from the root to a particular node forms a string.
# The idea is that all strings sharing common prefix should come from a common node
"""
Trie Nodes have the following characteristics:

  - Depth: Each node has a depth indicating its position in the string.
  - Counts: Nodes can store counts or information associated with the string that corresponds
    to the path from the root to that node.
  - Children: Each node can have children representing different characters.
"""

# The mismatch kernel is constructed using a Trie data structure for efficient matching of subsequences

In [ ]:
class trinode():
  """
  This class trinode represents a single node in the Trie data structure
  """
  def __init__(self, dpth = 0):
    self.dpth = dpth
    self.conts = {} #A dictionary that stores the counts of occurrences of sequences represented by this node
    self.children = defaultdict(self.create_) #A defaultdict where the keys are characters (e.g., 'A', 'C', 'G', 'T') and the values are TrieNode objects.

  def create_(self):
    return trinode(dpth = self.dpth + 1)

  def leaf_(self):
    return len(self.children)==0

  def __iter__(self):
    """
    This method allows for iteration over the Trie starting from the current node.
    """
    yield self  #When iterating over a TrieNode object, it yields itself first.

    #Now, iterating over its children and their grandchildren using a recursive loop.

    for chld in self.children.values():
      for gran_chld in chld:
        yield gran_chld
    """
    Iterating over all nodes in the Trie.
    """

In [ ]:
class tri:
  def __init__(self, len_sequence):
    self.rot = trinode() #creating the root node of the Trie using the trinode class.
    self.len_sequence = len_sequence


  def add_(self, seq_id, seq, n_mismatch): #This method adds a new sequence to the Trie.
    """
     - seq_id: Sequence identifier
     - seq: Sequence
     - n_mismatch: The number of mismatches associated with the sequence.
    """
    nod = self.rot

    for i in seq:
      nod = nod.childern[i]

    if seq_id not in nod.conts:
      nod.conts[seq_id] = n_mismatch
    else:
      nod.conts[seq_id] = min(nod.conts[seq_id], n_mismatch)

    @property
    def nods(self):
      for nod in self.rot:
        yield nod

    @property
    def nod_num(self):
      return sum(1 for i in self.nods)

    @property
    def lev(self):
      for nod in self.nods:
        if nod.leaf_():
          yield nod

    @property
    def lef_num(self):
      return sum(1 for i in self.lev)

In [ ]:
class mis_match_Kernel:
  def __init__(self, k_mer, n_mismatch=1, w=None):
    self.k_mer = k_mer
    self.w = w
    self.trie = tri(k_mer)
    self.n_mismatch = n_mismatch
    self.next_seq_id = 0
    self.seq_fitt = {}

    self.fitt = False
    self.fitt_on = None
    self.sequen = ['A','C','G','T']

    def fit(self, sequence):
      for i in tqdm(sequence):
        self.strng_fit(i)
      self.fitt = True
      self.fitt_on = sequence
      return self.build_k()

    def predict(self, m):
      for i in m:
        self.strng_fit(i)
      return self.build_k(m, self.fitt_on)

    def strng_fit(self, strn):
      if i in self.seq_fitt:
        return
      seq_id = self.next_seq_id
      self.next_seq_id +=1
      self.strng_fit[i] = seq_id

      for sub_ in self.substr(s, self.k_mer):
        for i in range (len (sub_)):
          for l in self.sequen:
            mis_match = l == sub_[i]
            sub_copy = sub_[:i] + l + sub_[i+1:]
            self.tri.add_(seq_id, sub_copy, int(mis_match))

    def build_k(self, m, sequ):
      m_id = [
          self.strng_fit[t] for t in m
      ]

      seque_id = [ self.strng_fit[i] for i in sequ]

      dot_ = defaultdict(float)
      sq_norm = defaultdict(float)

      w = self.w or 1.0
      set_m_id = set(m_id)
      set_sequ_id = set(seque_id)
      all = set_m_id | set_sequ_id

      a = dict(iterable=self.tri.leaf_, total = self.tri.lef_num)

      for N in tqdm(**a):
        N_id = set(N.conts.keys())

        for idx in N_id & all:
          sq_norm [idx] +=w**(2*N.conts[idx])

        for idx in N_id & set_m_id:
          for s_ in N_id & set_sequ_id:
            dot = w**(N.conts[idx])
            dot_[idx, s_] += dot

        K = np.zeros((len(m), len(sequ)))

        for a, i_idx in enumerate(m_id):
          for b, idx_ in enumerate(seque_id):
            K[a,b] = dot_[i_idx, idx_]/np.sqrt(sq_norm[i_idx]*sq_norm[idx_])
        return K


        @staticmethod

        def substr(s, k):
          return [s[a:a+k] for a in range(len(s)-k+1)]